# Emergency Planning

**Author**: Sebastian Granda

**Course**: Integer Programming

**References**
* Laporte, G., & Louveaux, F. V. (1993). The integer L-shaped method for stochastic integer programs with complete recourse.
* Birge, J. R., & Louveaux, F. (2011). Introduction to Stochastic Programming.

In this notebook we implement the L-shaped and Integer L-shaped methods in the iterative and callback versions using the multi-cut approach. The code can be found in the following [repository](https://github.com/SebastianGrandaA/EmergencyPlanning/).

To start, we use the following small instance to go through the process:

In [1]:
sample_instance = "EP_sites_5_scenarios_10_lf_0.5_seed_0";

## Implementation notes

This project has been implemented as a Julia package called `EmergencyPlanning`. The main entry-point is the `execute` function, which creates an `Input` instance, solves the problem, and returns the solution as a `Solution` object. As part of the nomenclature, **all the optimization methods have been implemented with the `solve` function name**, using the multiple dispatch technique: `solve(::Base)`, `solve(::LShaped, ::Iterative)`, `solve(::LShaped, ::Callback)`, `solve(::IntegerLShaped)`.

To set up this package, it is required to have `Gurobi.jl` installed with a valid license, otherwise replace `using Gurobi` and `Gurobi.Optimizer` from `src/EmergencyPlanning.jl` for an open source alternative, such as `GLPK.jl` :

```julia
using GLPK
solver = optimizer_with_attributes(GPLK.Optimizer)
```

All the necessary dependencies will be managed by running the following cell:

In [1]:
using Pkg
Pkg.activate(@__DIR__)
using EmergencyPlanning: benchmark!, test, execute, plot_summary!

  Activating project at `~/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning`


## Base model

The `Base` model allow us to compare the solutions obtained from the L-shaped methods against the original formulation of the problem description. Find this formulation at `src/methods/Base.jl`.

In [ ]:
base_solution = execute(model="Base", filename=sample_instance, limit=60, benchmark=true)

## L-shaped method

The L-shaped method is a decomposition technique for solving two-stage stochastic linear programs similar to Benders decomposition.
In the first stage the team location variables $x_{ik}$ are determined, while in the second stage the assignment of teams to sites $y^{s}_{ij}$ and the number of rescues $u^{s}_{ij}$ are determined for each scenario $s \in S$ independently.

The master problem provides candidate solutions and the subproblem either improves the lower bound (optimality cuts) or makes the problem feasible (feasibility cuts).
The master problem approximates the subproblem solutions by introducing an additional variable $\theta_{s} \forall s \in S$ that represents second-stage cost.
These cuts are added dynamically to the master problem using the dual solutions of the subproblems.

Given that the team location variables are fixed $\bar{x}_{ik}$, the subproblem solved to obtain the **optimality** cut is the following:

$$\text{SP-opt}^{s} = \quad \min - \sum_{i \in I} \sum_{j \in N(i)} u^{s}_{ij}$$

$$\text{s.t.} \quad \sum_{j \in N(i)} y^{s}_{ij} \leq 1 \quad \forall i \in I \quad \text{(exclusive assignment)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} \leq d^{s}_{i} \quad \forall i \in I \quad \text{(demand satisfaction)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} \leq \sum_{k \in K} C_{k} \bar{x}_{ik} \quad \forall i \in I \quad \text{(rescue capacity)}$$
$$u^{s}_{ij} \leq M y^{s}_{ij} \quad \forall i \in I, j \in N(i) \quad \text{(exclusive rescues)}$$
$$u^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$
$$y^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$

The objective function aims to maximize the total number of rescues by minimizing the negative number of rescues.
The exclusive assignment constraint ensures that rescue teams can be assigned to at most one neighboring site.
The demand satisfaction constraint guarantees that the number of rescues in each site does not exceed the demand.
The rescue capacity constraint provides an upper bound on the number of rescues in each site based on the capacity of the allocated teams.
The exclusive rescues constraint ensures that a team can rescue in a given site only if it is assigned to a neighboring site, using a large constant $M = \sum_{k \in K} C_{k}$.
The last two constraints define the domain of the variables $u^{s}_{ij}$ and $y^{s}_{ij}$, respectively.
Note that the integrality constraints of $y^{s}_{ij}$ has been relaxed to obtain the dual solutions.

Let $\lambda^{s}_{i}$ be the dual variable associated with the rescue capacity constraint, and define $v(s)$ as the objective value of the subproblem for scenario $s$.
As an assumption, we only consider the duals associated with the rescue capacity constraint, because it is the only constraint with relationship with the first-stage variables.
Then, the optimality cut for scenario $s$ is formulated as follows:

$$- \pi_{s} v(s) + \sum_{i \in I} \lambda^{s}_{i} \sum_{k \in K} C_{k} x_{ik} \leq \theta_{s}$$

Where $\pi_{s}$ is the probability of scenario $s$.
Due that the subproblem objective function is the only term in the master objective function, we add the optimality cut only if it improves the lower bound.
This cut is multplied by $-1$ to obtain the correct sign.

On the other hand, to obtain the **feasibility cut** we solve the auxiliary problem to determine the capacity deficit and surplus for each site $i \in I$.
Let $\delta^{+}_{i}$ and $\delta^{-}_{i}$ be the artificial variables associated with the capacity deficit and surplus, respectively.
Then, the auxiliary problem solved to obtain the feasibility cut is the following:

$$\text{SP-feas}^{s} = \quad \min \sum_{i \in I} \delta^{+}_{i} + \delta^{-}_{i}$$

$$\text{s.t.} \quad \sum_{j \in N(i)} y^{s}_{ij} \leq 1 \quad \forall i \in I \quad \text{(exclusive assignment)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} \leq d^{s}_{i} \quad \forall i \in I \quad \text{(demand satisfaction)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} + \delta^{+}_{i} - \delta^{-}_{i} \leq \sum_{k \in K} C_{k} \bar{x}_{ik} \quad \forall i \in I \quad \text{(rescue capacity)}$$
$$u^{s}_{ij} \leq M y^{s}_{ij} \quad \forall i \in I, j \in N(i) \quad \text{(exclusive rescues)}$$
$$u^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$
$$y^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$
$$\delta^{+}_{i} \geq 0 \quad \forall i \in I$$
$$\delta^{-}_{i} \geq 0 \quad \forall i \in I$$

The rescue capacity constraint now includes the artificial variables $\delta^{+}_{i}$ and $\delta^{-}_{i}$ to measure the infeasibility of the first-stage solution.
This auxiliary problem seeks to minimize the total capacity deficit and surplus, thus we consider the first-stage solution as feasible if all the artificial variables are zero.
Let $\beta^{s}_{i}$ be the dual variable associated with the rescue capacity constraint, and again consider $v(s)$ to represent the objective value for the feasibility subproblem in scenario $s$.
Then, the feasibility cut for scenario $s$ is formulated as follows:

$$- v(s) + \sum_{i \in I} \beta^{s}_{i} \sum_{k \in K} C_{k} x_{ik} \leq 0$$

The master problem is given by:

$$\text{(MP)} \quad \min - \sum_{s \in S} \pi_{s} \theta_{s}$$

$$\text{s.t.} \quad \sum_{k \in K} x_{ik} <= 1 \quad \forall i \in I \quad \text{(exclusive allocation)}$$
$$\sum_{k \in K} \sum_{i \in I} b_{k} x_{ik} \leq B \quad \text{(budget)}$$
$$\text{feasibility cuts}$$
$$\text{optimality cuts}$$
$$\theta_{s} \geq 0 \quad \forall s \in S$$
$$x_{ik} \in \{0, 1\} \quad \forall i \in I, k \in K$$

The objective function seeks to minimize the total expected recourse cost, which in the deterministic form is sum of the number of rescues in each scenario weighted by its probability.
The exclusive allocation constraint ensures that each team is allocated to at most one site.
The budget constraint ensures that the total cost of the team allocation does not exceed the budget limit $B$.

The multi-cut L-shaped method has been implemented in both the iterative and the callback versions.
The logic is orchestrated by the `solve!` function in either `src/methods/LShapedIterative.jl` or `src/methods/LShapedCallback.jl` files for each version. The master problem and subproblem formulations are located at `src/methods/problems/MasterProblem.jl` and `src/methods/problems/SubProblem.jl`, respectively, under the constructor functions named after them. In the master problem, there is a custom `solve!` function to ensure consistency of the iterations. In the subproblems, there are separate implementations for the optimality and feasibility subproblems.

For the iterative version, the subproblems are solved in parallel for each scenario; whereas for the callback version, the cuts are handled by the solver. The stop criteria for the iterative L-shaped method is based on the converage, which is the difference between the objective value of the master problem and the expected recourse, the maximum number of iterations and the maximum number of iterations without improvement.

To invoke the `solve` function of the iterative L-shaped method located at `src/methods/LShaped.jl`, we call:

In [8]:
ls_solution = execute(model="LShaped", :usage=:iterative, filename=sample_instance, limit=60, benchmark=true)

As expected, we obtain decimal solutions.

To use the L-shaped in the callback version:

In [ ]:
ls_solution_c = execute(model="LShaped", :usage=:callback, filename=sample_instance, limit=60, benchmark=true)

Explain solution.....

## Integer L-shaped method

Previouly, the integrality constraint of $y^{s}_{ij}$ has been relaxed.
In this section we explain the implementation of the Integer L-shaped method, which considers the integrality constraint. This method is implemented in the `src/methods/IntegerLShaped.jl` file, under the `solve` function, in the iterative version.

We start by solving the linear relaxation of the master problem. If the solution is integer, then we have found the optimal solution. Otherwise, we select a branching variable and create two nodes, one for each subproblem with the upper and lower bound constraints. This new subproblems are solved, evaluate if they should be pruned or not, and repeat the process until all nodes are pruned.

Therefore, the stop criteria is based on the quantity of pendant nodes and on the number of iterations. To determine whether a node should be pruned or not, we use three criteria: infeasibility, bound and integrality. In other words, we prune a node if it is infeasible or unbounded, or if the all variables are integer or if there exists a better solution (lower bound). The search criteria is breadth-first search, which means that we select the node with the best objective value over the set of pendant nodes. Finally, we use a random criteria to select the non-integer variable to branch on.

In [ ]:
ls_solution_c = execute(model="IntegerLShaped", :usage=:callback, filename=sample_instance, limit=60, benchmark=true)

## Numerical experiments

A `benchmark` function is created to execute the models with several instances and to compare them. The logic related to the benchmark process can be found at `src/services/benchmark.jl`.

In [4]:
SETTINGS = Dict{String, Dict}(
    "Base" => Dict(),
    "LShaped" => Dict(:usage => :iterative),
    # "LShaped" => Dict(:usage => :callback),
    "IntegerLShaped" => Dict(),
);
MODELS = collect(keys(SETTINGS));

In [5]:
SAMPLE_SIZE = -1 # -1 to run all

benchmark!(MODELS, SAMPLE_SIZE, SETTINGS)

┌ Info: Base | Instance EP_sites_100_scenarios_100_lf_0.2_seed_0 | Sites 100 | Teams 3 | Scenarios 100
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:71
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:18


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


SystemError: SystemError: opening file "/Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/outputs/solutions/EP_sites_100_scenarios_100_lf_0.2_seed_0_Base_allocations.csv": No such file or directory

## Single instance


In [ ]:
base_solution = execute(
    model="Base",
    filename="EP_sites_5_scenarios_10_lf_0.5_seed_0",
    limit=60,
    benchmark=true
)

# INTERESTING INSTAC E FOR INTEGR LSHAPED
# "EP_sites_50_scenarios_100_lf_0.5_seed_10"

In [ ]:
fieldnames(typeof(base_solution))

In [ ]:
base_solution.metrics

In [ ]:
ils_solution = execute(model="IntegerLShaped", filename="EP_sites_5_scenarios_10_lf_0.5_seed_0", limit=60, benchmark=true)

In [ ]:
ils_solution.metrics

## Test

In [ ]:
EXPECTED_RESULTS = Dict{String, Real}(
    "EP_sites_5_scenarios_10_lf_0.5_seed_0" => 10,
    "EP_sites_5_scenarios_100_lf_0.5_seed_10" => 10,
    "EP_sites_10_scenarios_50_lf_0.2_seed_10" => 10,
    # "EP_sites_5_scenarios_50_lf_0.5_seed_10" => 10,
    # "EP_sites_10_scenarios_10_lf_0.5_seed_0" => 20,
    # "EP_sites_10_scenarios_100_lf_0.5_seed_10" => 20,

    # "EP_sites_10_scenarios_1_lf_0.8_seed_10" => 60,
    # "EP_sites_50_scenarios_50_lf_0.5_seed_0" => 220,
    # "EP_sites_100_scenarios_100_lf_0.5_seed_0" => 500, # only L-shaped

    # "EP_sites_100_scenarios_10_lf_0.5_seed_0" => 500,
    # "EP_sites_100_scenarios_100_lf_0.8_seed_0" => 800, # [iterative] 58seg | [base] 1761seg



    # "EP_sites_5_scenarios_100_lf_0.2_seed_10", # TODO no feasible solution??
)

errors = test(MODELS, EXPECTED_RESULTS, SETTINGS)

In [ ]:
isempty(errors) || error("Errors found: $errors")

## Benchmark

In [ ]:
SAMPLE_SIZE = 1
benchmark!(MODELS, SAMPLE_SIZE, SETTINGS)

In [ ]:
INSTANCE_NAMES = [
    "EP_sites_5_scenarios_50_lf_0.5_seed_10",
    # "EP_sites_100_scenarios_100_lf_0.5_seed_10",
    # "EP_sites_100_scenarios_50_lf_0.5_seed_0",
    # "EP_sites_50_scenarios_100_lf_0.2_seed_0",
    # "EP_sites_100_scenarios_100_lf_0.8_seed_10"
]
benchmark!(MODELS, INSTANCE_NAMES, SETTINGS)

## After running the benchmark, we proceed analysing the results:

In [5]:
plot_summary!()

# TODO esto deberia ser GAP ! que se compute entre par soluciones de la misma instancia, y las estadisticas sean los gap.
# Ademas, solo min, avg, stg, max es suficiente
# Importante la agryupacion model instance no es suficiente (o relevante incluso) porque quiero saber el desempeno general no x instance.

┌ Info: Statistical Summary
│   stats = 131×10 DataFrame
 Row │ model_name         instance_name                      objective_value_min  objective_value_mean  objective_value_std  objective_value_max  execution_time_min  execution_time_mean  execution_time_std  execution_time_max 
     │ String31           String                             Float64              Float64               Float64              Float64              Float64             Float64              Float64             Float64            
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Base               EP_sites_100_scenarios_100_lf_0.…               200.0                 200.0                   NaN               200.0          20.7262              20.7262                     NaN         20.7262
   2 │ LShaped-iterative  EP_sites_100_s

ErrorException: Cannot convert DataFrames.DataFrame to series data for plotting